In [8]:
import pandas as pd
import numpy as np

In [9]:
# อ่านข้อมูล metadata ของ site ที่ต้องการดาวน์โหลด
site_df = pd.read_csv('../data/DeDe_49stations/DeDe_49sites_metadata.csv')

In [10]:
# merge ข้อมูลจาก CAMS, ERA5 และ DeDe เข้าด้วยกัน (ข้อมูลราย 15 นาที)
for i,row in site_df.iterrows():
    # Load
    cams_df = pd.read_csv(f'../data/CAMS/cams_time_series_15min/cams_time_series_15min_site{row['Site_id']:02d}.csv',parse_dates=['Datetime'])

    # Load ERA5 data
    era5_df = pd.read_csv(f'../data/ERA5/radiation/all_site_15min/era5_15min_2023_site{row['Site_id']:02d}.csv',parse_dates=['Datetime'])

    # Load DeDe data
    dede_df = pd.read_csv(f'../data/DeDe_49stations/merge_15min_all49sites/site_{row['Site_id']:02d}.csv',parse_dates=['Datetime'])

    # Merge dataframes ทั้งสามบน column 'Datetime' 
    merged_df = pd.merge(cams_df, era5_df, on='Datetime',how='outer')
    merged_df = pd.merge(merged_df, dede_df, on='Datetime',how='outer')

    # ลบ column ที่ไม่จำเป็นออก
    merged_df.drop(columns=[ 'TOA', 'Clear sky GHI', 'Clear sky BHI', 'Clear sky DHI',
        'Clear sky BNI', 'BHI', 'DHI', 'BNI', 'Reliability', 'Site_id', 'rawI',
        'CI_CM', 'CI_CM_interpolated','CI_RGB', 'CI_RGB_interpolated', 'Temperature', 'Relative Humidity',
        'Pressure', 'Wind speed', 'Wind direction', 'Rainfall', 'Snowfall',
        'Snow depth', 'Iclr'],
        inplace=True, errors='ignore')

    # เก็บข้อมูล
    merged_df.to_csv(f'../merged_data/solar_radiation_15min/solar_radiation_15min_site{row['Site_id']:02d}_2023.csv', index=False)

In [11]:
# merge ข้อมูลจาก CAMS, ERA5 และ DeDe เข้าด้วยกัน (ข้อมูลรายชั่วโมง)

# เตรียม dataframe สำหรับ complete datetime
st = '2023-01-01 00:00:00'
en = '2023-12-31 23:59:59'
time = pd.date_range(st, en, freq='1h')
for i,row in site_df.iterrows():
    # Load CAMS data
    cams_df = pd.read_csv(f'../data/CAMS/cams_time_series_1hr/cams_time_series_1hr_site{row['Site_id']:02d}.csv',parse_dates=['Datetime'])

    # Load ERA5 data
    era5_df = pd.read_csv(f'../data/ERA5/radiation/all_site_1hr/era5_1hr_2023_site{row['Site_id']:02d}.csv',parse_dates=['Datetime'])

    # Load DeDe data
    dede_df = pd.read_csv(f'../data/DeDe_49stations/merge_15min_all49sites/site_{row['Site_id']:02d}.csv',parse_dates=['Datetime'])
    # subsampling จากราย 15 นาทีมาเป็นรายชั่วโมง
    dede_df = dede_df.merge(pd.DataFrame({'Datetime': time}), on='Datetime', how='right')

    # Merge dataframes ทั้งสามบน column 'Datetime'
    merged_df = pd.merge(cams_df, era5_df, on='Datetime',how='outer')
    merged_df = pd.merge(merged_df, dede_df, on='Datetime',how='outer')

    # ลบ column ที่ไม่จำเป็นออก
    merged_df.drop(columns=[ 'TOA', 'Clear sky GHI', 'Clear sky BHI', 'Clear sky DHI',
        'Clear sky BNI', 'BHI', 'DHI', 'BNI', 'Reliability', 'Site_id', 'rawI',
        'CI_CM', 'CI_CM_interpolated','CI_RGB', 'CI_RGB_interpolated', 'Temperature', 'Relative Humidity',
        'Pressure', 'Wind speed', 'Wind direction', 'Rainfall', 'Snowfall',
        'Snow depth', 'Iclr'],
        inplace=True, errors='ignore')

    # เก็บข้อมูล
    merged_df.to_csv(f'../merged_data/solar_radiation_1hr/solar_radiation_1hr_site{row['Site_id']:02d}_2023.csv', index=False)

In [12]:
# merge ข้อมูลการ forecast CAMS กับข้อมูลจากเครื่องวัด (DeDe) เข้าด้วยกัน (ข้อมูลรายชั่วโมง)

# เตรียม dataframe สำหรับ complete datetime
st = '2023-01-01 00:00:00'
en = '2023-12-31 23:59:59'
time = pd.date_range(st, en, freq='1h')
for i,row in site_df.iterrows():
    # Load CAMS_forcast data
    cams_df = pd.read_csv(f'../data/CAMS/cams_forecast/all_site_1hr/cams_forecast_1hr_2023_site{row['Site_id']:02d}.csv',parse_dates=['Datetime'])
    # เพิ่มข้อมูลรายชั่วโมงที่ขาดไป
    cams_df = cams_df.merge(pd.DataFrame({'Datetime': time}), on='Datetime', how='left')

    # Load DeDe data
    dede_df = pd.read_csv(f'../data/DeDe_49stations/merge_15min_all49sites/site_{row['Site_id']:02d}.csv',parse_dates=['Datetime'])

    # Merge dataframes ทั้งสองบน column 'Datetime'
    merged_df = pd.merge(cams_df, dede_df, on='Datetime',how='outer')

    # ลบ column ที่ไม่จำเป็นออก
    merged_df.drop(columns=['Site_id','CI_CM',
        'CI_CM_interpolated', 'CI_RGB', 'CI_RGB_interpolated', 
        'Relative Humidity', 'Pressure', 
        'Rainfall', 'Snowfall', 'Snow depth', 'Short-wave irradiation','rawI'], inplace=True)

    # Save merged data
    merged_df.to_csv(f'../merged_data/cams_forecast_1hr/cams_forecast_1hr_site{row['Site_id']:02d}_2023.csv', index=False)

In [13]:
# merge ข้อมูลการ forecast CAMS กับข้อมูลจากเครื่องวัด (DeDe) เข้าด้วยกัน (ข้อมูลรายชั่วโมง)

# เตรียม dataframe สำหรับ complete datetime
st = '2023-01-01 00:00:00'
en = '2023-12-31 23:59:59'
time = pd.date_range(st, en, freq='15min')
for i,row in site_df.iterrows():
    # Load CAMS_forcast data
    cams_df = pd.read_csv(f'../data/CAMS/cams_forecast/all_site_15min/cams_forecast_15min_2023_site{row['Site_id']:02d}.csv', parse_dates=['Datetime'])
    # เพิ่มข้อมูลรายชั่วโมงที่ขาดไป
    cams_df = cams_df.merge(pd.DataFrame({'Datetime': time}), on='Datetime', how='right')

    # Load DeDe data
    dede_df = pd.read_csv(f'../data/DeDe_49stations/merge_15min_all49sites/site_{row['Site_id']:02d}.csv',parse_dates=['Datetime'])

    # Merge dataframes ทั้งสองบน column 'Datetime'
    merged_df = pd.merge(cams_df, dede_df, on='Datetime',how='outer')

    # ลบ column ที่ไม่จำเป็นออก
    merged_df.drop(columns=['Site_id','CI_CM',
        'CI_CM_interpolated', 'CI_RGB', 'CI_RGB_interpolated',
        'Relative Humidity', 'Pressure', 
        'Rainfall', 'Snowfall', 'Snow depth', 'Short-wave irradiation','rawI'], inplace=True)
    
    # Save merged data
    merged_df.to_csv(f'../merged_data/cams_forecast_15min/cams_forecast_15min_site{row['Site_id']:02d}_2023.csv', index=False)

In [14]:
# merge ข้อมูลตัวแปรเมฆจาก ERA5 กับข้อมูลจากเครื่องวัด (DeDe) เข้าด้วยกัน (ข้อมูลรายชั่วโมง)

# เตรียม dataframe สำหรับ complete datetime
st = '2023-01-01 00:00:00'
en = '2023-12-31 23:59:59'
time = pd.date_range(st, en, freq='15min')
for i,row in site_df.iterrows():

    # Load cloud era5
    era5_df = pd.read_csv(f'../data/ERA5/cloud/all_site_15min/era5_cloud_15min_2023_site{row['Site_id']:02d}.csv', parse_dates=['Datetime'])

    # Load DeDe data
    dede_df = pd.read_csv(f'../data/DeDe_49stations/merge_15min_all49sites/site_{row['Site_id']:02d}.csv',parse_dates=['Datetime'])

    # Merge dataframes on 'Datetime'
    dede_df = dede_df.merge(pd.DataFrame({'Datetime': time}), on='Datetime', how='right')
    merged_df = pd.merge(era5_df, dede_df, on='Datetime',how='outer')

    # คำนวณ clear sky index ด้วยค่ารังสีจาก DeDe
    merged_df['CSI'] = merged_df['I']/merged_df['Iclr']

    # ลบ column ที่ไม่จำเป็นออก
    merged_df.drop(columns=[ 'Site_id','I', 'rawI', 'CI_CM', 'CI_CM_interpolated',
        'CI_RGB', 'Temperature', 'Relative Humidity',
        'Pressure', 'Wind speed', 'Wind direction', 'Rainfall', 'Snowfall',
        'Snow depth', 'Short-wave irradiation','Iclr'], inplace=True)
    
    # Save merged data
    merged_df.to_csv(f'../merged_data/era5_cloud_15min/era5_cloud_15min_site{row['Site_id']:02d}_2023.csv', index=False)